# Testing Deployment

In [1]:
from dotenv import load_dotenv

load_dotenv()

import sys
import os

sys.path.append(os.getenv('SRC_DIR'))

from utils.system import display_system_info

display_system_info(markdown=True)


**Last Updated**: 2025-04-16 16:29:59

**Python Version**: 3.11.5  
**OS**: Windows 10.0.26100  
**Architecture**: 64bit  
**Hostname**: ShenLaptop  
**Processor**: Intel64 Family 6 Model 186 Stepping 3, GenuineIntel  
**RAM Size**: 15.65 GB  
  
        

In [2]:
from ultralytics import YOLO
import cv2
import numpy as np
import pygame

from collections import deque

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Using Programmatic Input

In [3]:
SIREN_PATH = os.path.join(os.getenv('DATA_DIR'), 'audio', 'siren.mp3')
PERSON_CLASS_ID = 0
DROWNING_CLASS_ID = 2
NUM_CLASSES = 3
HISTORY_LENGTH = 30
DROWNING_THRESHOLD = 15
SENSITIVITY = 0.2  # Sensitivity factor for smoothing, can be dynamically adjusted

def create_class_vector(confidence, class_id, num_classes=80):
    class_vector = np.zeros(num_classes)
    class_vector[class_id] = confidence
    return class_vector

def apply_temporal_smoothing(existing_vector, new_vector, sensitivity=SENSITIVITY):
    if existing_vector is None:
        # If there's no previous value, simply return the new vector
        return new_vector
    else:
        # Apply Exponential Moving Average (EMA) to stabilize
        return sensitivity * existing_vector + (1 - sensitivity) * new_vector

def get_stabilized_object_label(existing_label_vec, confidence, cls, sensitivity=SENSITIVITY, num_classes=80):
    new_label_vec = create_class_vector(confidence, cls, num_classes)
    
    # Apply temporal smoothing to stabilize the label
    return apply_temporal_smoothing(existing_label_vec, new_label_vec, sensitivity)

def get_class_from_label(label_vec):
    return int(np.argmax(label_vec))

def get_stabilized_class_predictions(existing_label_vec, class_confs, sensitivity=SENSITIVITY):
    # Convert class confidence list to a numpy array
    new_label_vec = np.array(class_confs)
    
    # Apply temporal smoothing to stabilize the class predictions
    return apply_temporal_smoothing(existing_label_vec, new_label_vec, sensitivity)

def is_activate_drowning_alert(tracked_info):
    for id, info in tracked_info.items():
        if len(info['frame_history']) < HISTORY_LENGTH:
            continue

        most_common = max(set(info['frame_history']), key=info['frame_history'].count)
        is_drowning = (
            most_common == DROWNING_CLASS_ID and 
            info['frame_history'].count(DROWNING_CLASS_ID) >= DROWNING_THRESHOLD and 
            get_class_from_label(info['obj_label_vec']) == PERSON_CLASS_ID
        )

        if is_drowning:
            return True
        
    return False

def simulate_action_class_prediction():
    randint = np.random.randint(DROWNING_CLASS_ID, NUM_CLASSES)
    return [1 if i == randint else 0 for i in range(NUM_CLASSES)]

def play_siren():
    if not pygame.mixer.music.get_busy():
        pygame.mixer.music.play()

def stop_siren():
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()

In [8]:
pygame.mixer.init()
pygame.mixer.music.load(SIREN_PATH)

yolo_model = YOLO(os.path.join(os.getenv('YOLO_DIR'), 'yolo11n.pt'))

filename = os.path.join(os.getenv('DATA_DIR'), 'videos', 'indoorpool.mp4')
cap = cv2.VideoCapture(filename)

tracked_info = {}

while cap.isOpened():
    drowning_alert = False
    ret, frame = cap.read()

    if not ret:
        print("End of video or cannot read the frame.")
        break

    result = yolo_model.track(frame, persist=True, tracker="botsort.yaml", verbose=False)[0]

    if result.boxes and result.boxes.id is not None:
        boxes = result.boxes.xyxy.cpu().numpy()
        obj_clses = result.boxes.cls.cpu().numpy()
        obj_confs = result.boxes.conf.cpu().numpy()
        ids = result.boxes.id.cpu().numpy()

        for box, cls, conf, id in zip(boxes, obj_clses, obj_confs, ids):
            if id not in tracked_info:
                existing_obj_label_vec = None
                existing_class_prediction_vec = None
                frame_history = deque(maxlen=HISTORY_LENGTH)
            else:
                existing_obj_label_vec = tracked_info[id]['obj_label_vec']
                existing_class_prediction_vec = tracked_info[id]['class_prediction_vec']
                frame_history = tracked_info[id]['frame_history']

            obj_label_vec = get_stabilized_object_label(existing_obj_label_vec, conf, int(cls))
            obj_label = get_class_from_label(obj_label_vec)

            if obj_label == PERSON_CLASS_ID:
                class_confs = simulate_action_class_prediction() # Simulate class prediction for demonstration purposes
                class_prediction_vec = get_stabilized_class_predictions(existing_class_prediction_vec, class_confs, SENSITIVITY)
                class_prediction = get_class_from_label(class_prediction_vec)

                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Person {id}, Predicted Class: {class_prediction}, Conf: {class_prediction_vec[class_prediction]}, obj_conf: {obj_label_vec[obj_label]}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
                frame_history.append(class_prediction)
            else:
                # print(f"Skipping object with ID {id} and label {obj_label} due to low confidence or not person, obj_conf{obj_label_vec[obj_label]}.")
                class_prediction_vec = existing_class_prediction_vec
                class_prediction = get_class_from_label(class_prediction_vec)

            tracked_info[id] = {
                'obj_label_vec': obj_label_vec,
                'class_prediction_vec': class_prediction_vec,
                'frame_history': frame_history
            }

        drowning_alert = is_activate_drowning_alert(tracked_info)

        if drowning_alert:
            print("Drowning alert!")
            play_siren()
        
    fps = cap.get(cv2.CAP_PROP_FPS)
    cv2.putText(frame, f'FPS: {fps:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.imshow(filename, frame)

    if not drowning_alert:
        stop_siren()
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
stop_siren()
cv2.destroyAllWindows()

# Relabelling for YOLO Training

In [3]:
import os
import shutil

yolo_training_data_dir = os.getenv('YOLO_TRAINING_DATA_DIR')
raw_data_dir = os.getenv('RAW_DATA_DIR')
img_dir = os.getenv('IMG_DIR')
label_dir = os.getenv('LABEL_DIR')
train_dir = os.getenv('TRAIN_DIR')
val_dir = os.getenv('VAL_DIR')
yolo_config_dir = os.getenv('YOLO_CONFIG_DIR')

In [ ]:
for split in [train_dir, val_dir]:
    img_src_dir = os.path.join(raw_data_dir, img_dir, split)
    label_src_dir = os.path.join(raw_data_dir, label_dir, split)

    img_dst_dir = os.path.join(yolo_training_data_dir, img_dir, split)
    label_dst_dir = os.path.join(yolo_training_data_dir, label_dir, split)

    os.makedirs(img_dst_dir, exist_ok=True)
    os.makedirs(label_dst_dir, exist_ok=True)

    for filename in os.listdir(img_src_dir):
        if filename.endswith(('.jpg', )):
            shutil.copy2(os.path.join(img_src_dir, filename), os.path.join(img_dst_dir, filename))

            label_filename = os.path.splitext(filename)[0] + '.txt'
            label_src_path = os.path.join(label_src_dir, label_filename)
            label_dst_path = os.path.join(label_dst_dir, label_filename)

            if os.path.exists(label_src_path):
                with open(label_src_path, 'r') as f:
                    lines = f.readlines()
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        parts[0] = '0'  # change class ID to 0
                        new_lines.append(' '.join(parts))
                with open(label_dst_path, 'w') as f:
                    f.writelines(line + '\n' for line in new_lines)
            else:
                # If no label exists, create an empty label file (optional)
                open(label_dst_path, 'w').close()


# YOLO Fine-Tuning

In [4]:
yaml_content = (
f"""# Dataset paths
path: {yolo_training_data_dir}
train: {os.path.join(img_dir, train_dir)}
val: {os.path.join(img_dir, val_dir)}

# Classes
nc: 1
names: ['person']
""")

yaml_path = os.path.join(yolo_config_dir, "fine_tuning.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)
print(f"Created data configuration at {yaml_path}")

Created data configuration at C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\config\yolo\fine_tuning.yaml


In [ ]:
from ultralytics import YOLO
import torch

model = YOLO(os.path.join(os.getenv('YOLO_DIR'), 'yolo11n.pt'))

results = model.train(
    data=yaml_path,
    epochs=2,
    patience=10,
    batch=16,
    imgsz=640,
    save=True,
    save_period=1,
    cache='disk',
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=8,
    project=os.getenv('YOLO_DIR'),
    name='yolo11n-finetuned',
    exist_ok=True,
    optimizer='auto',
    seed=32,
    deterministic=True,
    single_cls=True,
    resume=False,
    fraction=0.01,
    dropout=0.1,
    val=True,
    plots=True,
    degrees=10,
    mosaic=1
)

Default image size: 640
New https://pypi.org/project/ultralytics/8.3.109 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.94  Python-3.11.5 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
engine\trainer: task=detect, mode=train, model=C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n.pt, data=C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\config\yolo\fine_tuning.yaml, epochs=2, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=1, cache=disk, device=cpu, workers=8, project=C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO, name=yolo11n-finetuned, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=32, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=0.01, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=

train: Scanning C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\yolo_training\labels\train... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<00:00, 144.92it/s]

train: New cache created: C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\yolo_training\labels\train.cache



train: Caching images (0.8GB Disk): 100%|██████████| 70/70 [00:00<00:00, 7052.47it/s]
val: Scanning C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\yolo_training\labels\val.cache... 1572 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1572/1572 [00:00<?, ?it/s]
val: Caching images (17.9GB Disk): 100%|██████████| 1572/1572 [00:00<00:00, 4103.68it/s]


Plotting labels to C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n-finetuned\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n-finetuned
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        2/2         0G      1.791      3.421      1.288         17        640: 100%|██████████| 5/5 [00:43<00:00,  8.63s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [03:44<00:00,  4.49s/it]


                   all       1572       2317    0.00487      0.992      0.659      0.298

2 epochs completed in 0.172 hours.
Optimizer stripped from C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n-finetuned\weights\last.pt, 5.4MB
Optimizer stripped from C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n-finetuned\weights\best.pt, 5.4MB

Validating C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n-finetuned\weights\best.pt...
Ultralytics 8.3.94  Python-3.11.5 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [03:18<00:00,  3.96s/it]


                   all       1572       2317    0.00487      0.992       0.66      0.298
Speed: 1.6ms preprocess, 84.9ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\models\YOLO\yolo11n-finetuned


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002741E8A21D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480